## 基本思路

|**步骤**|任务|说明|方法|
|:----:|:----:|:-----|:----:|
|**1**|**生成网址**|分析<mark>网址规律</mark>，批量生成网址|for循环，format函数|
|2|请求+获取网页数据|模拟人工打开网页，并将网页存储为数据对象|requests包|
|**3**|**解析数据**|分析数据规律，整理出所需字段|<mark>pyquery包</mark>，json包<br>（还有<mark>lxml包和beautifulsoup4包</mark>）|
|4|存储数据|使用csv包将数据存储到csv文件中|csv包|
|5|批量爬取|对所有网址循环步骤2-4|for循环|

# 生成网址

## 分析网址规律

1. **```任务对象```**：
    - https://book.douban.com/top250<br>
<br>
<br>
2. **```基本情况```**：
    - 每翻一页，网址中的某个**<u>```数字发生规律性变化```</u>**
    - 每页25本书的信息，一共10页

3. **```规律```**：
- **<u>```原网址```</u>**：
    * https://book.douban.com/top250?start=0
    * https://book.douban.com/top250?start=25
    * https://book.douban.com/top250?start=50

    * ……
    * https://book.douban.com/top250?start=225

- ```总结规律```：
    - 第1页，数字为0
    - 第2页，数字为25
    - 第3页，数字为50
    - ……
    - 第10页，数字为225
<br>
<br>
    - <mark><big>第p页，数字为(p-1)*25</big></mark>

## 批量生成网址

In [1]:
# 尝试用format替换数字
'https://book.douban.com/top250?start={num}'.format(num=100)
# 'https://book.douban.com/top250?start={num}'.format(num=0)   # num取0

'https://book.douban.com/top250?start=100'

In [40]:
'https://book.douban.com/top250?start={num}'.format(num=25)  # num取25

'https://book.douban.com/top250?start=25'

In [ ]:
# 简写
template = 'https://book.douban.com/top250?start={num}'
template.format(num=25)

In [1]:
# 循环生成网址，并存入网址列表中
url_list = []    # 生成空列表，用于存储网址
template = 'https://book.douban.com/top250?start={num}'
for p in range(1,11):   # range取1到10
    url = template.format(num=(p-1)*25)    # 取1时，为(1-1)*25，即num为0，以此类推
    url_list.append(url)   # 将上面获得的url【添加进】列表

In [2]:
# 查看url_list情况（获得10页的网址）
url_list

['https://book.douban.com/top250?start=0',
 'https://book.douban.com/top250?start=25',
 'https://book.douban.com/top250?start=50',
 'https://book.douban.com/top250?start=75',
 'https://book.douban.com/top250?start=100',
 'https://book.douban.com/top250?start=125',
 'https://book.douban.com/top250?start=150',
 'https://book.douban.com/top250?start=175',
 'https://book.douban.com/top250?start=200',
 'https://book.douban.com/top250?start=225']

In [1]:
# 函数：生成网址  
# 返回值：url_list为网址列表
def generate_url():   # 添加：定义函数名
    url_list = []
    template = 'https://book.douban.com/top250?start={num}'
    for p in range(1,11):
        url = template.format(num=(p-1)*25)
        url_list.append(url)
    return url_list        # 添加：返回网址列表

In [2]:
# 调用函数  generate_url
url_list_tmp = generate_url()
url_list_tmp

['https://book.douban.com/top250?start=0',
 'https://book.douban.com/top250?start=25',
 'https://book.douban.com/top250?start=50',
 'https://book.douban.com/top250?start=75',
 'https://book.douban.com/top250?start=100',
 'https://book.douban.com/top250?start=125',
 'https://book.douban.com/top250?start=150',
 'https://book.douban.com/top250?start=175',
 'https://book.douban.com/top250?start=200',
 'https://book.douban.com/top250?start=225']

# 请求+获取网页数据	

- 以第1页为例，来请求并获取网页数据
- 例子：https://book.douban.com/top250?start=0

In [3]:
import requests   # 导入requests包

url = 'https://book.douban.com/top250'
resp = requests.get(url)   # 用get向服务器请求获取数据
resp # 查看状态码，返回码418说明访问不成功，需要伪装访问，假装是人工打开网页

<Response [418]>

##  1. **状态码**：


    * 1开头：信息状态码
    * 2开头：成功状态码
    * 3开头：重定向状态码
    * 4开头：客户端错误状态码
    * 5开头：服务端错误状态码

In [9]:
resp.text

''

## 2. **```应对方式```**：

加入请求头headers，一般包括user-agent、cookie和referer等


- ```user-agent```：浏览器类型及版本、操作系统及版本、浏览器内核等的信息标识
- ```cookie```：用于识别用户身份、记录历史的一段数据（浏览器访问服务器后，服务器传给浏览器的）
- ```referer```：告诉服务器该网页是从哪个页面链接过来的

In [4]:
# 尝试加入user-agent
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'}
resp = requests.get(url, headers=headers)  # 设置参数headers
resp #返回码200说明访问成功

<Response [200]>

In [5]:
# 查看返回内容，搜索该页面的“红楼梦”是否在返回的文本中，确认与网页一致
resp.text

'\n\n<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="ua-windows ua-webkit book-new-nav">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <title>豆瓣读书 Top 250</title>\n  \n<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\\s+|\\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call

In [6]:
# 将返回内容放入html中，html为str
html = resp.text
html
type(html)

str

In [19]:
# 函数：获得html  get_html(url)
# 参数说明：url为单个网址
# 返回值：html为网址的html数据，即网页源代码的字符串
def get_html(url):         # 添加：定义函数名
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0'}
    resp = requests.get(url,headers=headers)
    html = resp.text
    return html            # 添加：返回网址的html数据

In [80]:
# 调用函数get_html(url)
url_tmp = 'https://book.douban.com/top250?start=0'  # 第1页
html_tmp = get_html(url_tmp)
html_tmp

'\n\n<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="ua-windows ua-webkit book-new-nav">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <title>豆瓣读书 Top 250</title>\n  \n<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\\s+|\\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call

# 解析数据

- 上面获得的html数据是杂乱的，需要把它**整理成结构化的数据**，即解析数据
- 方式：用开发者工具中的elements，分析字段的分布情况，找出具有**定位唯一性的标签**

In [20]:
from pyquery import PyQuery  #导入pyquery包


In [ ]:

doc = PyQuery(html) # 将html字符串转换为pyquery数据，便于解析
type(doc)   # 数据类型为pyquery.pyquery.PyQuery
doc.text()    # 数据变“干净”了

In [82]:
# 只要书籍信息：调用items获得生成器
doc.items('tr.item')    # 书的定位标签为tr.item

<generator object PyQuery.items at 0x0000029169C37370>

In [83]:
for book in doc.items('tr.item'):
#     print(type(book))   # 有25本
    print(book)       # 查看这25本书的信息

<tr class="item">
          <td width="100" valign="top">
            <a class="nbg" href="https://book.douban.com/subject/1007305/" onclick="moreurl(this,{i:'0'})">
              <img src="https://img1.doubanio.com/view/subject/s/public/s1070959.jpg" width="90"/>
            </a>
          </td>
          <td valign="top">
            
            <div class="pl2">


              <a href="https://book.douban.com/subject/1007305/" onclick="&quot;moreurl(this,{i:'0'})&quot;" title="红楼梦">
                红楼梦

                
              </a>



                  <img src="/pics/read.gif" alt="可试读" title="可试读"/>

              
            </div>

              <p class="pl">[清] 曹雪芹 著 / 人民文学出版社 / 1996-12 / 59.70元</p>

            

              
              <div class="star clearfix">
                  <span class="allstar50"/>
                  <span class="rating_nums">9.6</span>

                <span class="pl">(
                    413885人评价
                )</span>
          

```三种解析情况```：
1. <u>```根据定位标签，直接解析获得```</u>，例如用book('.pl2 a').text()获得“红楼梦”
2. 用第一种方法获得数据，再进行<u>```list元素提取```</u>，例如获得['[清] 曹雪芹 著', ' 人民文学出版社 ', ' 1996-12 ', ' 68.00元']，分别提取作者、出版社、出版时间、价格等4个字段
3. 用第一种方法获得数据，再用<u>```正则表达式提取```</u>，例如获得“(158378人评价)”，提取“158378（数字）”作为“评价人数（字段）”

<center><font size=4><b>情况1：直接解析</b></font></center>

In [84]:
for book in doc.items('tr.item'):
#     print(book('.info h2 a'))
#     print(book('.pl2 a').attr('title'))   #取书名的两种方式：一是通过attr方法
#     print(book('.pl2 a').text())          # 二是通过text方法，相对完整
    book_name = book('.pl2 a').text()     # 放入book_name字段中
    print(book_name)

红楼梦
活着
1984
三体全集 : 地球往事三部曲
哈利·波特
百年孤独
飘
三国演义（全二册）
房思琪的初恋乐园
动物农场
福尔摩斯探案全集（上中下）
白夜行
小王子
撒哈拉的故事
安徒生童话故事集
天龙八部
呐喊
悉达多 : 一首印度的诗
杀死一只知更鸟
邓小平时代
明朝那些事儿（1-9） : 限量版
失踪的孩子 : 那不勒斯四部曲4
新名字的故事 : 那不勒斯四部曲2
沉默的大多数 : 王小波杂文随笔全编
野草


In [85]:
for book in doc.items('tr.item'):
    print(book('.pl2 a').text())          

红楼梦
活着
1984
三体全集 : 地球往事三部曲
哈利·波特
百年孤独
飘
三国演义（全二册）
房思琪的初恋乐园
动物农场
福尔摩斯探案全集（上中下）
白夜行
小王子
撒哈拉的故事
安徒生童话故事集
天龙八部
呐喊
悉达多 : 一首印度的诗
杀死一只知更鸟
邓小平时代
明朝那些事儿（1-9） : 限量版
失踪的孩子 : 那不勒斯四部曲4
新名字的故事 : 那不勒斯四部曲2
沉默的大多数 : 王小波杂文随笔全编
野草


In [86]:
# 用类似方法的字段有：book_name, desc, score
for book in doc.items('tr.item'):
    book_name = book('.pl2 a').text()
    desc = book('span.inq').text()
    score = book('span.rating_nums').text()
    print(book_name, desc, score)

红楼梦 都云作者痴，谁解其中味？ 9.6
活着 生的苦难与伟大 9.4
1984 栗树荫下，我出卖你，你出卖我 9.4
三体全集 : 地球往事三部曲 地球往事三部曲 9.5
哈利·波特 从9¾站台开始的旅程 9.7
百年孤独 魔幻现实主义文学代表作 9.3
飘 革命时期的爱情，随风而逝 9.3
三国演义（全二册） 是非成败转头空 9.3
房思琪的初恋乐园 向死而生的文学绝唱 9.2
动物农场 太阳底下并无新事 9.3
福尔摩斯探案全集（上中下） 名侦探的代名词 9.3
白夜行 一宗离奇命案牵出跨度近20年步步惊心的故事 9.2
小王子 献给长成了大人的孩子们 9.1
撒哈拉的故事 游荡的自由灵魂 9.2
安徒生童话故事集 为了争取未来的一代 9.3
天龙八部 有情皆孽，无人不冤 9.2
呐喊 新文学的第一声呐喊 9.2
悉达多 : 一首印度的诗  9.3
杀死一只知更鸟 有一种东西不能遵循从众原则，那就是——人的良心 9.2
邓小平时代 个人命运背后的历史变局 9.3
明朝那些事儿（1-9） : 限量版 不拘一格的历史书写 9.2
失踪的孩子 : 那不勒斯四部曲4 我的整个生命，只是一场为了提升社会地位的低俗斗争。 9.2
新名字的故事 : 那不勒斯四部曲2 探索青年时代的激情、困惑、挣扎、背叛和失去 9.2
沉默的大多数 : 王小波杂文随笔全编 沉默是沉默者的通行证 9.1
野草 我以这一丛野草，在明与暗，生与死，过去与未来之际，献于友与仇，人与兽，爱者与不爱者之前作证。 9.5


<center><font size=4><b>情况2：提取列表元素</b></font></center>

In [87]:
for book in doc.items('tr.item'):
#     info_list = book('.info .pub').text()    # 用左下划线'/'分隔的字符串
    info_list = book('p.pl').text().split('/')     # 以'/'为分隔符split，得到info_list列表
#     print(info_list)
#     print(info_list[-3])    # 用[]提取列表元素，例如倒数第3个是出版社，索引为-3
    publisher = info_list[-3].strip()   # 以同样的方式获得authors, publisher, pub_time, price字段
#     print(publisher)
    pub_time = info_list[-2]
    price = info_list[-1]
    authors = ','.join(info_list[:-3])   # 将多个作者元素，组合到一个字符串里
#     print(authors)
    print(authors, publisher, pub_time, price)

[清] 曹雪芹 著  人民文学出版社  1996-12   59.70元
余华  作家出版社  2012-8-1   20.00元
[英] 乔治·奥威尔 , 刘绍铭  北京十月文艺出版社  2010-4-1   28.00
刘慈欣  重庆出版社  2012-1   168.00元
J.K.罗琳 (J.K.Rowling) , 苏农  人民文学出版社  2008-12-1   498.00元
[哥伦比亚] 加西亚·马尔克斯 , 范晔  南海出版公司  2011-6   39.50元
[美国] 玛格丽特·米切尔 , 李美华  译林出版社  2000-9   40.00元
[明] 罗贯中  人民文学出版社  1998-05   39.50元
林奕含  北京联合出版公司  2018-2   45.00元
[英] 乔治·奥威尔 , 荣如德  上海译文出版社  2007-3   10.00元
[英] 阿·柯南道尔 , 丁钟华 等 , 群众出版社  1981-8  53.00元 68.00元
[日] 东野圭吾 , 刘姿君  南海出版公司  2013-1-1   39.50元
[法] 圣埃克苏佩里 , 马振聘  人民文学出版社  2003-8   22.00元
三毛  哈尔滨出版社  2003-8   15.80元
（丹麦）安徒生 , 叶君健  人民文学出版社  1997-08   25.00元
金庸  生活·读书·新知三联书店  1994-5   96.00元
鲁迅  人民文学出版社  1973-3   0.36元
[德] 赫尔曼·黑塞 , 姜乙  天津人民出版社  2017-1   32.00元
[美] 哈珀·李 , 高红梅  译林出版社  2012-9   32.00元
【美】傅高义 (Ezra.F.Vogel) , 冯克利  生活·读书·新知三联书店  2013-1-18   88.00元
当年明月  中国海关出版社  2009-4   358.20元
[意] 埃莱娜·费兰特 , 陈英  人民文学出版社  2018-7   62.00元
[意] 埃莱娜·费兰特 , 陈英  人民文学出版社  2017-4   59.00元
王小波  中国青年出版社  1997-10   27.00元
鲁迅  人民文学出版社  1973-3   0.20元


<center><font size=4><b>情况3：正则表达式提取</b></font></center>

In [22]:
import re    # 导入re包

In [ ]:

for book in doc.items('tr.item'):
    people_num_raw = book('span.pl').text()
#     print(people_num_raw)
#     print(re.findall('[0-9]+', people_num_raw))    # 匹配people_num_raw字段中的数字，获得列表
#     print(re.findall('[0-9]+', people_num_raw)[0])   # 取列表中的第1个元素，索引为0
    people_num = re.findall('[0-9]+', people_num_raw)[0]  # 放入people_num字段中
    print(people_num)

1. 将这些```字段组合成字典，放入列表```中
2. 书名、作者、出版社等作为```key```，人类简史、尤瓦尔·赫拉利、中信出版社等作为```value```
3. ```例子```如下：

In [89]:
bookinfo_list_tmp = [
    {'book_name': '人类简史：从动物到上帝', 'author': '尤瓦尔·赫拉利', 'publisher': '中信出版社'},
    {'book_name': '理想国', 'author': '柏拉图', 'publisher': '商务印书馆'},
    {'book_name': '社会契约论', 'author': '卢梭', 'publisher': '商务印书馆'}
    ]
print(bookinfo_list_tmp)
print('数据类型为：',type(bookinfo_list_tmp))   # 类型为list
print('数据长度为：',len(bookinfo_list_tmp))    # 长度为3，即3本书的信息

[{'book_name': '人类简史：从动物到上帝', 'author': '尤瓦尔·赫拉利', 'publisher': '中信出版社'}, {'book_name': '理想国', 'author': '柏拉图', 'publisher': '商务印书馆'}, {'book_name': '社会契约论', 'author': '卢梭', 'publisher': '商务印书馆'}]
数据类型为： <class 'list'>
数据长度为： 3


In [23]:
bookinfo_list = []   # 生成空列表，用于存储书籍信息
doc = PyQuery(html)
for book in doc.items('tr.item'):
    book_name = book('.pl2 a').text()    # 情况1：直接解析
    desc = book('span.inq').text()
    score = book('.rating_nums').text()
    
    info_list = book('p.pl').text().split('/')    # 情况2：提取列表元素
    publisher = info_list[-3].strip()   
    pub_time = info_list[-2]
    price = info_list[-1]
    authors = ''.join(info_list[:-3]) 
    
    people_num_raw = book('span.pl').text()       # 情况3：正则表达式提取
    people_num = re.findall('[0-9]+', people_num_raw)[0]
    
    bookinfo = {'book_name':book_name,   # 为每本书创建一个字典，不同字段建构不同键值对
                 'authors':authors,
                 'publisher':publisher,
                 'pub_time':pub_time,
                 'desc':desc,
                 'score':score,
                 'people_num':people_num,
                 'price':price,
                }
    
    bookinfo_list.append(bookinfo)      # 将字典添加进bookinfo_list列表中

In [52]:
bookinfo_list
# len(bookinfo_list)

[{'book_name': '顾城的诗',
  'authors': '顾城 ',
  'publisher': '人民文学出版社',
  'pub_time': ' 1998-12 ',
  'desc': '火焰是我们诗歌唯一的读者',
  'score': '8.8',
  'people_num': '26009',
  'price': ' 20.00'},
 {'book_name': '会饮篇',
  'authors': '[古希腊] 柏拉图  王太庆 ',
  'publisher': '商务印书馆',
  'pub_time': ' 2013-1 ',
  'desc': '',
  'score': '9.3',
  'people_num': '11008',
  'price': ' 12.00元'},
 {'book_name': '众病之王 : 癌症传',
  'authors': '[美] 悉达多·穆克吉  李虎 ',
  'publisher': '中信出版社',
  'pub_time': ' 2013-2 ',
  'desc': '癌症传',
  'score': '9.0',
  'people_num': '12983',
  'price': ' 42.00元'},
 {'book_name': '火星救援',
  'authors': '[美]安迪•威尔  陈灼 ',
  'publisher': '译林出版社',
  'pub_time': ' 2015-10 ',
  'desc': '跟火星来一场不是你死就是我活的过家家游戏',
  'score': '8.9',
  'people_num': '19457',
  'price': ' 38.00'},
 {'book_name': '父权制与资本主义',
  'authors': '[日] 上野千鹤子  邹韵 ',
  'publisher': '浙江大学出版社',
  'pub_time': ' 2020-3 ',
  'desc': '',
  'score': '9.2',
  'people_num': '16503',
  'price': ' 58.00元'},
 {'book_name': '上学记',
  'authors': '何兆武 口

In [10]:
# 函数：解析数据  extract_bookinfo_list(html)
# 参数说明：html为网页源代码的字符串
# 返回值：bookinfo_list为书籍的字典列表
def extract_bookinfo_list(html):         # 添加：定义函数名
    bookinfo_list = []
    doc = PyQuery(html)
    for book in doc.items('tr.item'):
        try:                             # 添加：try语句，避免特殊网页中断整个循环
            book_name = book('.pl2 a').text()    
            desc = book('span.inq').text()
            score = book('.rating_nums').text()
            info_list = book('p.pl').text().split('/')    
            publisher = info_list[-3].strip()   
            pub_time = info_list[-2]
            price = info_list[-1]
            authors = ''.join(info_list[:-3])
            people_num_raw = book('span.pl').text()       
            people_num = re.findall('[0-9]+', people_num_raw)[0]

            bookinfo = {'book_name':book_name,
                         'authors':authors,
                         'publisher':publisher,
                         'pub_time':pub_time,
                         'desc':desc,
                         'score':score,
                         'people_num':people_num,
                         'price':price,
                        }

            bookinfo_list.append(bookinfo)
        except:                        # 添加：except和pass语句，如果碰到bug，那么跳出此次循环、不执行任何操作，进行下一次循环
            pass
        
    return bookinfo_list               # 添加：返回书籍的字典列表

In [37]:
html

'\n\n<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="ua-windows ua-webkit book-new-nav">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <title>豆瓣读书 Top 250</title>\n  \n<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\\s+|\\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call

In [57]:
# 调用函数extract_bookinfo_list(html)
bookinfo_list = extract_bookinfo_list(html)
print(bookinfo_list)
print('书籍数量为：', len(bookinfo_list))

[{'book_name': '顾城的诗', 'authors': '顾城 ', 'publisher': '人民文学出版社', 'pub_time': ' 1998-12 ', 'desc': '火焰是我们诗歌唯一的读者', 'score': '8.8', 'people_num': '26009', 'price': ' 20.00'}, {'book_name': '会饮篇', 'authors': '[古希腊] 柏拉图  王太庆 ', 'publisher': '商务印书馆', 'pub_time': ' 2013-1 ', 'desc': '', 'score': '9.3', 'people_num': '11008', 'price': ' 12.00元'}, {'book_name': '众病之王 : 癌症传', 'authors': '[美] 悉达多·穆克吉  李虎 ', 'publisher': '中信出版社', 'pub_time': ' 2013-2 ', 'desc': '癌症传', 'score': '9.0', 'people_num': '12983', 'price': ' 42.00元'}, {'book_name': '火星救援', 'authors': '[美]安迪•威尔  陈灼 ', 'publisher': '译林出版社', 'pub_time': ' 2015-10 ', 'desc': '跟火星来一场不是你死就是我活的过家家游戏', 'score': '8.9', 'people_num': '19457', 'price': ' 38.00'}, {'book_name': '父权制与资本主义', 'authors': '[日] 上野千鹤子  邹韵 ', 'publisher': '浙江大学出版社', 'pub_time': ' 2020-3 ', 'desc': '', 'score': '9.2', 'people_num': '16503', 'price': ' 58.00元'}, {'book_name': '上学记', 'authors': '何兆武 口述 ', 'publisher': '生活·读书·新知三联书店', 'pub_time': ' 2006-8 ', 'desc': '20世纪中国知识分子

# 存储数据

In [13]:
import csv     # 导入csv包


In [ ]:

# 打开文件
file = open('./data/books_250.csv', 'a+', encoding='utf-8', newline='') # 文件名为books_philosophy.csv，模式为a+，如果已有文件，在末尾追加，如果没，则生成新文件；编码为utf-8，需要区分换行符
fieldnames = ['book_name', 'authors', 'publisher', 'pub_time', 'desc', 'score', 'people_num', 'price']  # 设置标题行fieldnames
writer = csv.DictWriter(file, fieldnames=fieldnames)    # 要求以字典的形式写入数据，fieldnames注明字典中键的名称，也就是书名、作者这些标题名
writer.writeheader()  # 将fieldnames设置的标题key写入首行

# 循环写入字典列表：因为有很多本书，需要一行行写入
for bookinfo in bookinfo_list:
    writer.writerow(bookinfo)  # 写入一行书籍信息
    
file.close() # 关闭文件

# 批量爬取

刚才我们做了几项工作：
1. 生成网址————发现网址规律，获得所有网址url_list
2. 请求+获取网页数据————获得html网页源代码字符串（**单个网址**）
3. 解析数据————从html中获得书籍的各字段信息，存储为字典列表（**单个网址**）
4. 存储数据（**单个网址**）
<br>
<br>
<br>
<b><mark>下面对所有网址url_list循环步骤2-4</mark><b>

|**步骤**|任务|函数|输入参数|返回值|
|:----:|:-----|:-----|:-----|:-----|
|1|生成网址|**|--|url_list网址列表|
|**<mark>2</mark>**|**<mark>请求+获取网页数据</mark>**|get_html(url)|单个网址|html网页源代码的字符串|
|**<mark>3</mark>**|**<mark>解析数据</mark>**|extract_bookinfo_list(html)|（单个网址）html网页源代码的字符串|bookinfo_list书籍的字典列表|
|**<mark>4</mark>**|**<mark>存储数据</mark>**|--|--|--|
|5|批量爬取|--|--|--|

In [44]:
url_list = []
template = 'https://book.douban.com/top250?start={num}'
for p in range(1,11):   # range取1到11
    url = template.format(num=(p-1)*25)
    url_list.append(url)
url_list

['https://book.douban.com/top250?start=0',
 'https://book.douban.com/top250?start=25',
 'https://book.douban.com/top250?start=50',
 'https://book.douban.com/top250?start=75',
 'https://book.douban.com/top250?start=100',
 'https://book.douban.com/top250?start=125',
 'https://book.douban.com/top250?start=150',
 'https://book.douban.com/top250?start=175',
 'https://book.douban.com/top250?start=200',
 'https://book.douban.com/top250?start=225']

In [46]:
# 打开文件
file = open('./data/books_250.csv', 'a+', encoding='utf-8', newline='') 
fieldnames = ['book_name', 'authors', 'publisher', 'pub_time', 'desc', 'score', 'people_num', 'price'] 
writer = csv.DictWriter(file, fieldnames=fieldnames)    
writer.writeheader() 

# 对所有网址url_list循环步骤2-4
for url in url_list:
    
    html = get_html(url)       # 【步骤2：得到html数据，用函数get_html(url)】
    
    bookinfo_list = extract_bookinfo_list(html)  # 【步骤3：整理成结构化数据，用函数extract_bookinfo_list(html)】
    
    for bookinfo in bookinfo_list:      # 【步骤4：写数据到csv文件】
        writer.writerow(bookinfo)
    
file.close() # 关闭文件

In [24]:
# 函数：爬虫主函数  main(filename)
# 参数说明：filename为文件名称
# 仅执行命令，不返回任何值
def main(filename):         # 添加：定义函数名
    print('开始采集豆瓣读书top 250书籍！')              # 添加：说明“开始采集豆瓣读书top 250书籍！”
    
    # 生成所有网址url_list
    url_list = generate_url()
    
    # 打开文件
    file = open(filename, 'a+', encoding='utf-8', newline='')     # 修改：将文件名称，设定为函数的参数
    fieldnames = ['book_name', 'authors', 'publisher', 'pub_time', 'desc', 'score', 'people_num', 'price'] 
    writer = csv.DictWriter(file, fieldnames=fieldnames)    
    writer.writeheader() 

    # 对所有网址url_list循环步骤2-4
    for url in url_list:
        print('正在采集：{url}'.format(url=url))    # 添加：说明“正在采集<url>”
        html = get_html(url)
        bookinfo_list = extract_bookinfo_list(html)
        for bookinfo in bookinfo_list:
            writer.writerow(bookinfo)

    file.close()

    print('采集完毕！')            # 添加：说明“采集完毕！”

In [25]:
main(filename='./data/books_250.csv')

开始采集豆瓣读书top 250书籍！
正在采集：https://book.douban.com/top250?start=0
正在采集：https://book.douban.com/top250?start=25
正在采集：https://book.douban.com/top250?start=50
正在采集：https://book.douban.com/top250?start=75
正在采集：https://book.douban.com/top250?start=100
正在采集：https://book.douban.com/top250?start=125
正在采集：https://book.douban.com/top250?start=150
正在采集：https://book.douban.com/top250?start=175
正在采集：https://book.douban.com/top250?start=200
正在采集：https://book.douban.com/top250?start=225
采集完毕！


# 参考：
- [Excel打开监控数据CSV文件乱码如何处理？_云监控服务 CES_常见问题_产品咨询_华为云 ](https://support.huaweicloud.com/ces_faq/ces_faq_0018.html)